In [ ]:
# Link - https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

In [1]:
%pip install langchain

  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.32-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl.metadata (32 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached orjson-3.10.7-cp311-none-win_amd64.whl.metadata (51 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp311-none-win_amd64.whl.metadata (6.7 kB)
  Using cached greenlet-3.0.3-cp311-cp311-win_amd64.whl.met

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# -- Langsmith tracking --
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [2]:
# +++++++++++++ Open AI ++++++++++++++

# load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56be9999-d02e-4277-ad2b-d7ddc8dc2638-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [4]:
# --- Using StrOutputParser to retrieve the output from the AIMessage ---

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
# result = model.invoke(messages)

# parser.invoke(result)

chain = model | parser
chain.invoke(messages)

'Ciao!'

In [6]:
# +++++++++++ Prompt Templates ++++++++++++

from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
  [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "Italian", "text": "Hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Italian:'), HumanMessage(content='Hi')])

In [7]:
result.to_messages()

[SystemMessage(content='Translate the following into Italian:'),
 HumanMessage(content='Hi')]

In [8]:
chain = prompt_template | model | parser
chain.invoke({"language": "Italian", "text": "Hi"})

'Ciao'

In [9]:
# ---- Client side ------

from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8080/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it